In this project, I'll be working with a dataset of used cars from eBay Kleinanzeigen, a [classifieds](https://en.wikipedia.org/wiki/Classified_advertising) section of the [German eBay](https://data.world/data-society/used-cars-data) website.

A few modifications have been made from the original dataset:

50,000 data points from the full dataset have been sampled, to ensure the code runs quickly in the hosted environment.
The dataset has been dirtied a bit to more closely resemble what you would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with).
Data dictionary for the dataset:

    dateCrawled - When the ad was first crawled. All field-values are taken from this date.
    name - Name of the car.
    seller - Whether the seller is private or a dealer.
    offerType - The type of listing.
    price - The listed selling price of the car.
    abtest - Whether the listing is included in an A/B test.
    vehicleType - The type of vehicle.
    yearOfRegistration - The year in which the car was first registered.
    gearbox - The type of transmission.
    powerPS - The power of the car in PS.
    model - The car model name.
    odometer - How many kilometers the car has driven.
    monthOfRegistration - The month in which the car was first registered.
    fuelType - What type of fuel the car uses.
    brand - The brand of the car.
    notRepairedDamage - If the car has a damage which is not yet repaired.
    dateCreated - The date the eBay listing was created.
    nrOfPictures - The number of pictures in the ad.
    postalCode - The postal code for the location of the vehicle.
    lastSeenOnline - When the crawler saw this ad last online.

The aim of this project is to clean the dataset and perform some initial analysis on it

In [30]:
import pandas as pd
import numpy as np
autos = pd.read_csv("autos.csv", encoding = "latin1")

In [31]:
autos
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


From the displayed data above, the following are observations:

    The dataset contains 20 columns, most of which are strings.
    Some columns have null values, but none have more than ~20% null values.
    The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.
Converting the column names from camelcase to snakecase and changing some of the column names based on the data dictionary will make the data to be more descriptive.

In [32]:
print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [33]:
mapping_dict = {
"yearOfRegistration" : "registration_year",
"monthOfRegistration" : "registration_month",
"notRepairedDamage" : "unrepaired_damage",
"dateCreated" : "ad_created",
"dateCrawled" : "date_crawled",
"name" : "name",
"seller" : "seller",
"offerType" : "offer_type",
"price" : "price",
"abtest" : "ab_test",
"vehicleType" : "vehicle_type",
"powerPS" : "power_PS",
"model" : "model",
"odometer" : "odometer",
"fuelType" : "fuel_type",
"brand" : "brand",
"nrOfPictures" : "num_photos",
"postalCode" : "postal_code",
"lastSeen" : "last_seen"
}
autos.columns = autos.columns.map(mapping_dict)
print(autos.columns)


Index([      'date_crawled',               'name',             'seller',
               'offer_type',              'price',            'ab_test',
             'vehicle_type',  'registration_year',                  nan,
                 'power_PS',              'model',           'odometer',
       'registration_month',          'fuel_type',              'brand',
        'unrepaired_damage',         'ad_created',         'num_photos',
              'postal_code',          'last_seen'],
      dtype='object')


In [34]:
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,NaN,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-29 23:42:13,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Initial observations:

- There are a number of text columns where all (or nearly all) of the values are the same:
    - `seller`
    - `offer_type`
- The `num_photos` column looks odd, we'll need to investigate this further.

In [35]:
autos["num_photos"].value_counts()

num_photos
0    50000
Name: count, dtype: int64

mIt looks like the `num_photos` column has `0` for every column.  We'll drop this column, plus the other two we noted as mostly one value.

In [36]:
autos = autos.drop(["num_photos", "seller", "offer_type"], axis=1)

There are two columns, `price` and `auto`, which are numeric values with extra characters being stored as text, it needs to be cleaned and converted these.

In [37]:
autos["price"] = (autos["price"].str.replace("$","").str.replace(",","").astype(int))
autos["price"].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

In [38]:
autos["odometer"] = (autos["odometer"].str.replace("km","").str.replace(",","").astype(int))
autos["odometer"].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer, dtype: int64